## Using *checkpoint_schedules*

This example aims to introduce the usage of *checkpoint_schedules* through an initial illustration of how this package works and how to interpret the run time of forward and adjoint solvers using a schedule and generators provided by *checkpointing_schedules* package. 

### Managing the forward and adjoint executions
Let us consider the `CheckpointingManager` class, which plays the essential role in managing forward and adjoint executions. This management is given by iterating over a schedule with the execution of `CheckpointingManager.execute(cp_schedule)`. To perform this task, the method requires a `cp_schedule` argument, which must be a *checkpoint_schedules* object containing a schedule attribute and a generator method. This generator is responsible for yielding the *checkpoint_schedules* actions to be executed.

Whitin `CheckpointingManager.execute`, we implement the *checkpointing_schedules* actions using single-dispatch functions. The base function, `action`, is decorated with the `singledispatch`. Specific actions functions are then established by using the register method of the `action`. Hence, `CheckpointingManager.execute` effectively calls the specific action according to a schedule.

In [1]:
from checkpoint_schedules import Forward, EndForward, Reverse, Copy, Move, EndReverse, StorageType
import functools

class CheckpointingManager():
    """Manage the executions of the forward and adjoint solvers.

    Attributes
    ----------
    max_n : int
        Total steps used to execute the solvers.
    chk_ram : int, optional
        Number of checkpoint stored on `'RAM'`.
    chk_disk : int, optional
        Number of checkpoint stored on `'DISK'`.
    """
    def __init__(self, max_n, chk_ram=0, chk_disk=0):
        self.max_n = max_n
        self.save_ram = chk_ram
        self.save_disk = chk_disk
        self.list_actions = []
        
    def execute(self, cp_schedule):
        """Execute forward and adjoint with a checkpointing schedule.

        Parameters
        ----------
        cp_schedule : CheckpointSchedule
            Checkpoint schedule object.
        """
        @functools.singledispatch
        def action(cp_action):
            raise TypeError("Unexpected action")

        @action.register(Forward)
        def action_forward(cp_action):
            def illustrate_runtime(a, b, singlestorage):
                if singlestorage:
                    print(((a + '\u2212\u2212' + b)*(n1-cp_action.n0)).rjust(n1*4) +
                   "   "*(self.max_n - n1 + 2) + 
                   self.list_actions[len(self.list_actions) - 1])
                else:
                    print((a + ('\u2212\u2212\u2212' + b)*(n1-cp_action.n0)).rjust(n1*4) +
                    "   "*(self.max_n - n1 + 2) + 
                    self.list_actions[len(self.list_actions) - 1])

            nonlocal model_n
            n1 = min(cp_action.n1, self.max_n)
            if cp_action.write_ics and cp_action.write_adj_deps:
                singlestorage = True
                a = '\u002b'
                b = '\u25b6'
            else:
                singlestorage = False
                if cp_action.write_ics:
                    a = '\u002b'
                else:
                    a = ''
                if cp_action.write_adj_deps:
                    b = "\u25b6"
                else:
                    b = "\u25b7"
            illustrate_runtime(a, b, singlestorage)
            
            model_n = n1
            if n1 == self.max_n:
                # Imposing the latest time step.
                # It is required for the online schedule.
                cp_schedule.finalize(n1)

        @action.register(Reverse)
        def action_reverse(cp_action):
            nonlocal model_r
            print((('\u25c0' + '\u2212\u2212\u2212')*(cp_action.n1-cp_action.n0)).rjust(cp_action.n1*4) 
                  + "   "*(self.max_n - cp_action.n1 + 2) + 
                    self.list_actions[len(self.list_actions) - 1])
            model_r += cp_action.n1 - cp_action.n0
            
        @action.register(Copy)
        def action_copy(cp_action):
            print("    "*(self.max_n + 1) + 
                    self.list_actions[len(self.list_actions) - 1])

        @action.register(Move)
        def action_move(cp_action):
            print("    "*(self.max_n + 1) + 
                    self.list_actions[len(self.list_actions) - 1])

        @action.register(EndForward)
        def action_end_forward(cp_action):
            assert model_n == self.max_n
            # The correct number of adjoint steps has been taken
            print("End Forward" + "   "*(self.max_n) + 
                    self.list_actions[len(self.list_actions) - 1])
            if cp_schedule._max_n is None:
                cp_schedule._max_n = self.max_n
            
        @action.register(EndReverse)
        def action_end_reverse(cp_action):
            nonlocal model_r
            assert model_r == self.max_n
            print("End Reverse" + "   "*(self.max_n) + 
                  self.list_actions[len(self.list_actions) - 1])

        model_n = 0
        model_r = 0

        for count, cp_action in enumerate(cp_schedule):
            self.list_actions.append(str(cp_action))
            action(cp_action)
            if isinstance(cp_action, EndReverse):  
                break

### Schedule for no adjoint computation

The code below creates the solver manager object given by the class `CheckpointManager`. In this example, the adjoint computation is not performed and there is no the storage of the forward data. Therefore, only maximum number of steps (representing `max_n`) is the required argument. 

In [2]:
max_n = 4 # Total number of time steps.
solver_manager = CheckpointingManager(max_n) # manager object

The `NoneCheckpointSchedule` provides the schedule and the generator to execute only the forward solver without storage the forward data.

In [3]:
from checkpoint_schedules import NoneCheckpointSchedule
cp_schedule = NoneCheckpointSchedule() # Checkpoint schedule object
solver_manager.execute(cp_schedule) # Execute the forward solver by following the schedule.

−−−▷−−−▷−−−▷−−−▷      Forward(0, 9223372036854775807, False, False, <StorageType.NONE: None>)
End Forward            EndForward()


The output of the `solver_manager.execute(cp_schedule)` execution illustrates the time-steps run of the forward solver on the left side, while displaying *checkpoint_schedules* actions on the right side. 

The general of the *Forward* action is as follows:
```python
Forward(n0, n1, write_ics, write_adj_deps, storage_type)
```
which reads as follows:
- Advance the forward solver from step `n0` to the start of any step `n1`.
- `write_ics` and `write_adj_deps` are booleans that indicate whether the forward 
solver should store the forward restart data and the forward data required for the 
adjoint computation, respectively.
- `storage_type` is an enum that indicates the type of storage required for the
forward restart data and the forward data required for the adjoint computation.

In the following, we have *Forward* action in the context of the *NoneCheckpointSchedule* schedule.
```python
Forward(0, 9223372036854775807, False, False, <StorageType.NONE: None>)
```
- Advance the forward solver from step `n0 = 0` to the start of any step `n1`.
- Both `write_ics` and `write_adj_deps` are  set to `'False'`, indicating that the forward solver does not store the forward restart data and the forward data required for the adjoint computation. 
- The storage type is `StorageType.NONE`, indicating that no specific storage type is required. 
- The `EndForward()` action indicates that the forward solver has reached the end of the time interval.

This schedule is referred to as online since it does not require specifying a value for the maximum steps to obtain the schedule. Hence, users can define any desired step as needed. For this specific case, we set the maximum step with `max_n = 4`, which is an attribute of the `CheckpointingManager`. Whe then inform the schedule the final step by executing the following code below into the `action_forward` function.
```python
 cp_schedule.finalize(n1)
```
The `n1` value is set according the user's requirement. 

Another action shown in the current schedule is the `EndForward()` action, which indicates the forward solver has reached the end of the time interval.

### Schedule for storing all time-step forward data

We now begin to present the schedules when there is the adjoint solver computation. 

The following code is valuable for the cases where the user intend to store the forward data for all time-steps and does not applies any checkpointing strategy, e. g., given by the Revolve approach [1]. 

The schedule in which there is the storage of the forward data for all time steps in `'RAM`' is reached by using the `SingleMemoryStorageSchedule` class. The code below shows how to employ this type of schedule in the forward and adjoint computations.

In this schedule there is no storage of the forward restart data based on the concept that when no checkpointing strategy is employed, there exists no necessity for retaining forward data restart.

In [4]:
from checkpoint_schedules import SingleMemoryStorageSchedule

cp_schedule = SingleMemoryStorageSchedule()
solver_manager.execute(cp_schedule)

−−−▶−−−▶−−−▶−−−▶      Forward(0, 9223372036854775807, False, True, <StorageType.ADJ_DEPS: 3>)
End Forward            EndForward()
◀−−−◀−−−◀−−−◀−−−      Reverse(4, 0, True)
End Reverse            EndReverse(False,)


In this particular case, the *Forward* action Is given by:
```python
Forward(0, 9223372036854775807, False, True, <StorageType.RAM: 0>)
```
which reads:
- Advance the forward solver from the step `n0=0` to the start of any step `n1`.
- Do not store the forward restart data once if `write_ics` is `'False'`.
- Store the forward data required for the adjoint computation once `write_adj_deps` is `'True'`.
- Storage type is `<StorageType.ADJ_DEPS: 3>`, which indicates the storage in a place that holds the forward data required for the adjoint computation.

In general, the *Reverse* action is given by:
```python
Reverse(n0, n1, clear_adj_deps)
```
which reads:
- Advance the adjoint solver from the step `n0` to the start of the step `n1`.
- Clear the adjoint dependency data if `clear_adj_deps` is `'True'`.

In the currrent case, the *Reverse* action reads:
```python
Reverse(4, 0, True)
```
-  Advance the forward solver from the step `4` to the start of the step `0`.
- Clear the adjoint dependency (forward data) once `clear_adj_deps` is `'True'`.

If the adjoint computation is done, we have the action *EndReverse(True)* to indicate that the reverse action reached the end.

*checkpoint_schedules* also allows the user execute forward and adjoint solver by storing all adjoint dependency in the `'disk'`. This type of schedule is generate by the class `SingleDiskStorageSchedule`. The following code illustrates the run of the forward and adjoint solver foir this case.

In [5]:
from checkpoint_schedules import SingleDiskStorageSchedule

cp_schedule = SingleDiskStorageSchedule()
solver_manager.execute(cp_schedule)


−−−▶−−−▶−−−▶−−−▶      Forward(0, 9223372036854775807, False, True, <StorageType.DISK: 1>)
End Forward            EndForward()
                    Copy(4, <StorageType.DISK: 1>, <StorageType.ADJ_DEPS: 3>)
            ◀−−−      Reverse(4, 3, True)
                    Copy(3, <StorageType.DISK: 1>, <StorageType.ADJ_DEPS: 3>)
        ◀−−−         Reverse(3, 2, True)
                    Copy(2, <StorageType.DISK: 1>, <StorageType.ADJ_DEPS: 3>)
    ◀−−−            Reverse(2, 1, True)
                    Copy(1, <StorageType.DISK: 1>, <StorageType.ADJ_DEPS: 3>)
◀−−−               Reverse(1, 0, True)


In the case illustrated above, forward and adjoint executions with `SingleDiskStorageSchedule` also has the *Copy* action, which indicates copying of the forward data from a storage type
to another.  Hence, it is required to copy the forward data from the `'disk'` to a place that holds the forward data used for the adjoint computation.

In general, the *Copy* action is given by:
```python
Copy(n, from_storage, to_storage)
```
which reads:
- Copy the data required for the adjoint computation from the step `n`.
- The term `from_storage` denotes the storage type responsible for retaining forward data at step n, while `to_storage` refers to the designated storage type for storing this forward data.


Thus, taking one of the *Copy* actions from the example above:
```python
Copy(4, <StorageType.DISK: 1>, <StorageType.ADJ_DEPS: 3>)
```
which reads:
- Copy the data required for the adjoint computation from the step `4`.
- The forward data is copied from `'disk'`, and the storage type to copy (`StorageType.ADJ_DEPS`) indicates the place that holds the forward data required for the adjoint computation.

The provided schedule implies that the forward data required for the adjoint computation remains accessible in the `'disk'` storage. If the user wants to move the data from one storage type to another, and consequently the data is not more available in the original storage type, the optional `move_data` argument within the `SingleDiskStorageSchedule` must be configured as `True`, as demonstrated in the following example.

In [6]:
cp_schedule = SingleDiskStorageSchedule(move_data=True)
solver_manager.execute(cp_schedule)

−−−▶−−−▶−−−▶−−−▶      Forward(0, 9223372036854775807, False, True, <StorageType.DISK: 1>)
End Forward            EndForward()
                    Move(4, <StorageType.DISK: 1>, <StorageType.ADJ_DEPS: 3>)
            ◀−−−      Reverse(4, 3, True)
                    Move(3, <StorageType.DISK: 1>, <StorageType.ADJ_DEPS: 3>)
        ◀−−−         Reverse(3, 2, True)
                    Move(2, <StorageType.DISK: 1>, <StorageType.ADJ_DEPS: 3>)
    ◀−−−            Reverse(2, 1, True)
                    Move(1, <StorageType.DISK: 1>, <StorageType.ADJ_DEPS: 3>)
◀−−−               Reverse(1, 0, True)


The *Move* action has the general form:
```python
Move(n, from_storage, to_storage)
```
which reads:
- Move the data required for the adjoint computation from the step `n`.
- The term `from_storage` denotes the storage type responsible for retaining forward data at step `n`, while `to_storage` refers to the designated storage type for moving this forward data.


Thus, taking one of the *Copy* actions from the example above:
```python
Move(4, <StorageType.DISK: 1>, <StorageType.ADJ_DEPS: 3>)
```
which reads:
- Move the data required for the adjoint computation from the step `4`.
- The forward data is moved from `'disk'` to a storage used for the adjoint computation.

### Schedules given by checkointing methods

Now, let us consider the schedules given by the checkpointing strategies. We begin by exploring the Revolve approach, according to introduced in reference [1].

The Revolve checkpointing strategy  generate schedule with storage only in `'RAM'`. So, we now set the attribute `chk_ram` of the `CheckpointingManager` class to specify the number of steps at whichshould store the forward restart data. 

The schedule is generated by the `Revolve` class, which takes the number of steps at which the adjoint data should be stored in `'RAM'` as an argument. This number of steps is specified by the attribute `chk_ram`.

The code below shows how to generate a schedule for the `Revolve` checkpointing strategy and illustrates the execution of the forward and adjoint computations using the Revolve schedule.

In [14]:
from checkpoint_schedules import Revolve
chk_ram = 2 
solver_manager = CheckpointingManager(max_n, chk_ram=chk_ram) # manager object
cp_schedule = Revolve(max_n, chk_ram) 
solver_manager.execute(cp_schedule)

+−−−▷−−−▷            Forward(0, 2, True, False, <StorageType.RAM: 0>)
       +−−−▷         Forward(2, 3, True, False, <StorageType.RAM: 0>)
            −−−▶      Forward(3, 4, False, True, <StorageType.ADJ_DEPS: 3>)
End Forward            EndForward()
            ◀−−−      Reverse(4, 3, True)
                    Move(2, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
        −−−▶         Forward(2, 3, False, True, <StorageType.ADJ_DEPS: 3>)
        ◀−−−         Reverse(3, 2, True)
                    Copy(0, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
−−−▷               Forward(0, 1, False, False, <StorageType.FWD_RESTART: 2>)
    −−−▶            Forward(1, 2, False, True, <StorageType.ADJ_DEPS: 3>)
    ◀−−−            Reverse(2, 1, True)
                    Move(0, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
−−−▶               Forward(0, 1, False, True, <StorageType.ADJ_DEPS: 3>)
◀−−−               Reverse(1, 0, True)
End Reverse            EndReverse()


The employment of the checkpointing strategies in an adjoint-based gradient involves the forward solver restarting and recomputation.  As we see in the example above, we have the *Forward* action  as follows:
```python
Forward(0, 2, True, False, <StorageType.RAM: 0>)
```
which means:
- Advance from time step 0 to the start of the time step 2.
- Store the forward data required to restart the forward solver from time step 0.
- The storage of the forward restart data is done in RAM.

*In the time step illustrations, we have `'+−−−▷−−−▷'` associated to the *Forward* action explained above. The symbol `'+'` indicates that the forward data required for the forward recomputation from the step 0 is stored in `'RAM'`.*


The following code illustrates the forward and adjoint computations using the checkpointing given by H-Revolve strategy [2]. In this case, the storage of the forward data is done in`'RAM'` and on `'disk'`. The checkpointing schedule is generated with `HRevolve` class, which requires the following parameters: maximum steps stored in RAM (`snap_in_ram`), maximum steps stored on disk (`snap_on_disk`), and the number of time steps (`max_n`). Thus, we firs import the necessary module `HRevolve` from the *checkpoint_schedules* package. Then, `HRevolve` class is instantiated with the following parameters: `snap_in_ram = 1`, `snap_on_disk = 1`, and `max_n = 4`. Finally, it is shown an illutration of the forward and adjoint executions using the H-Revolve checkpointing.

In [15]:
from checkpoint_schedules import HRevolve
chk_disk = 1
chk_ram = 1
solver_manager = CheckpointingManager(max_n, chk_ram=chk_ram, chk_disk=chk_disk) # manager object
cp_schedule = HRevolve(max_n, chk_ram, snap_on_disk=chk_disk)
solver_manager.execute(cp_schedule)

+−−−▷−−−▷−−−▷         Forward(0, 3, True, False, <StorageType.RAM: 0>)
            −−−▶      Forward(3, 4, False, True, <StorageType.ADJ_DEPS: 3>)
End Forward            EndForward()
            ◀−−−      Reverse(4, 3, True)
                    Copy(0, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
−−−▷−−−▷            Forward(0, 2, False, False, <StorageType.FWD_RESTART: 2>)
        −−−▶         Forward(2, 3, False, True, <StorageType.ADJ_DEPS: 3>)
        ◀−−−         Reverse(3, 2, True)
                    Copy(0, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
−−−▷               Forward(0, 1, False, False, <StorageType.FWD_RESTART: 2>)
    −−−▶            Forward(1, 2, False, True, <StorageType.ADJ_DEPS: 3>)
    ◀−−−            Reverse(2, 1, True)
                    Move(0, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
−−−▶               Forward(0, 1, False, True, <StorageType.ADJ_DEPS: 3>)
◀−−−               Reverse(1, 0, True)
End Reverse            EndReverse()


The following examples generate *checkpoint_schedules* actions similar to the explanations given above. The schedules below refer to the following checkpointing methods:

- **Periodic Disk Revolve**: . 
- **Disk Revolve**: 
- **Two-Level**: 
- **Mixed Revolver**: 
- **Mulistage**: 

In [9]:
from checkpoint_schedules import PeriodicDiskRevolve, DiskRevolve, MixedCheckpointSchedule, MultistageCheckpointSchedule, TwoLevelCheckpointSchedule
print("------------------------------------------------")
print("Disk Revolve checkpointing schedule")
print("------------------------------------------------")
solver_manager = CheckpointingManager(max_n, chk_ram=chk_ram) # manager object``
cp_schedule = DiskRevolve(max_n, chk_ram)
solver_manager.execute(cp_schedule)

------------------------------------------------
Disk Revolve checkpointing schedule
------------------------------------------------
+−−−▷−−−▷−−−▷         Forward(0, 3, True, False, <StorageType.RAM: 0>)
            −−−▶      Forward(3, 4, False, True, <StorageType.ADJ_DEPS: 3>)
End Forward            EndForward()
            ◀−−−      Reverse(4, 3, True)
                    Copy(0, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
−−−▷−−−▷            Forward(0, 2, False, False, <StorageType.FWD_RESTART: 2>)
        −−−▶         Forward(2, 3, False, True, <StorageType.ADJ_DEPS: 3>)
        ◀−−−         Reverse(3, 2, True)
                    Copy(0, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
−−−▷               Forward(0, 1, False, False, <StorageType.FWD_RESTART: 2>)
    −−−▶            Forward(1, 2, False, True, <StorageType.ADJ_DEPS: 3>)
    ◀−−−            Reverse(2, 1, True)
                    Move(0, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
−−−▶            

In [10]:
print("------------------------------------------------")
print("Periodic disk revolve checkpointing schedule")
print("------------------------------------------------")
cp_schedule = PeriodicDiskRevolve(max_n, chk_ram)
solver_manager.execute(cp_schedule)

------------------------------------------------
Periodic disk revolve checkpointing schedule
------------------------------------------------
We use periods of size  3
+−−−▷−−−▷−−−▷         Forward(0, 3, True, False, <StorageType.RAM: 0>)
            −−−▶      Forward(3, 4, False, True, <StorageType.ADJ_DEPS: 3>)
End Forward            EndForward()
            ◀−−−      Reverse(4, 3, True)
                    Copy(0, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
−−−▷−−−▷            Forward(0, 2, False, False, <StorageType.FWD_RESTART: 2>)
        −−−▶         Forward(2, 3, False, True, <StorageType.ADJ_DEPS: 3>)
        ◀−−−         Reverse(3, 2, True)
                    Copy(0, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
−−−▷               Forward(0, 1, False, False, <StorageType.FWD_RESTART: 2>)
    −−−▶            Forward(1, 2, False, True, <StorageType.ADJ_DEPS: 3>)
    ◀−−−            Reverse(2, 1, True)
                    Move(0, <StorageType.RAM: 0>, <StorageTyp

In [11]:
print("------------------------------------------------")
print("Mixed checkpointing schedule")
print("------------------------------------------------")
cp_schedule = MixedCheckpointSchedule(max_n, chk_disk)
solver_manager.execute(cp_schedule)

------------------------------------------------
Mixed checkpointing schedule
------------------------------------------------
+−−−▷−−−▷−−−▷         Forward(0, 3, True, False, <StorageType.DISK: 1>)
            −−−▶      Forward(3, 4, False, True, <StorageType.ADJ_DEPS: 3>)
End Forward            EndForward()
            ◀−−−      Reverse(4, 3, True)
                    Copy(0, <StorageType.DISK: 1>, <StorageType.FWD_RESTART: 2>)
−−−▷−−−▷            Forward(0, 2, False, False, <StorageType.FWD_RESTART: 2>)
        −−−▶         Forward(2, 3, False, True, <StorageType.ADJ_DEPS: 3>)
        ◀−−−         Reverse(3, 2, True)
                    Move(0, <StorageType.DISK: 1>, <StorageType.FWD_RESTART: 2>)
−−−▶               Forward(0, 1, False, True, <StorageType.DISK: 1>)
    −−−▶            Forward(1, 2, False, True, <StorageType.ADJ_DEPS: 3>)
    ◀−−−            Reverse(2, 1, True)
                    Move(0, <StorageType.DISK: 1>, <StorageType.ADJ_DEPS: 3>)
◀−−−               Reverse(1, 

/home/ma/d/ddolci/work/checkpoint_schedules/checkpoint_schedules/mixed.py:59: RuntimeWarning: Numba not available -- using memoization
  warnings.warn("Numba not available -- using memoization",


In [12]:
print("------------------------------------------------")
print("Two-Level checkpointing schedule")
print("------------------------------------------------")
revolver = TwoLevelCheckpointSchedule(2, binomial_snapshots=2)
solver_manager.execute(revolver)

------------------------------------------------
Two-Level checkpointing schedule
------------------------------------------------
+−−−▷−−−▷            Forward(0, 2, True, False, <StorageType.DISK: 1>)
       +−−−▷−−−▷      Forward(2, 4, True, False, <StorageType.DISK: 1>)
End Forward            EndForward()
                    Copy(2, <StorageType.DISK: 1>, <StorageType.FWD_RESTART: 2>)
        −−−▷         Forward(2, 3, False, False, <StorageType.FWD_RESTART: 2>)
            −−−▶      Forward(3, 4, False, True, <StorageType.ADJ_DEPS: 3>)
            ◀−−−      Reverse(4, 3, True)
                    Copy(2, <StorageType.DISK: 1>, <StorageType.FWD_RESTART: 2>)
        −−−▶         Forward(2, 3, False, True, <StorageType.ADJ_DEPS: 3>)
        ◀−−−         Reverse(3, 2, True)
                    Copy(0, <StorageType.DISK: 1>, <StorageType.FWD_RESTART: 2>)
−−−▷               Forward(0, 1, False, False, <StorageType.FWD_RESTART: 2>)
    −−−▶            Forward(1, 2, False, True, <StorageTy

In [13]:
print("------------------------------------------------")
print("Multistage checkpointing schedule")
print("------------------------------------------------")
cp_schedule = MultistageCheckpointSchedule(max_n, 0, chk_disk)
solver_manager.execute(cp_schedule)

------------------------------------------------
Multistage checkpointing schedule
------------------------------------------------
+−−−▷−−−▷−−−▷         Forward(0, 3, True, False, <StorageType.DISK: 1>)
            −−−▶      Forward(3, 4, False, True, <StorageType.ADJ_DEPS: 3>)
End Forward            EndForward()
            ◀−−−      Reverse(4, 3, True)
                    Copy(0, <StorageType.DISK: 1>, <StorageType.FWD_RESTART: 2>)
−−−▷−−−▷            Forward(0, 2, False, False, <StorageType.FWD_RESTART: 2>)
        −−−▶         Forward(2, 3, False, True, <StorageType.ADJ_DEPS: 3>)
        ◀−−−         Reverse(3, 2, True)
                    Copy(0, <StorageType.DISK: 1>, <StorageType.FWD_RESTART: 2>)
−−−▷               Forward(0, 1, False, False, <StorageType.FWD_RESTART: 2>)
    −−−▶            Forward(1, 2, False, True, <StorageType.ADJ_DEPS: 3>)
    ◀−−−            Reverse(2, 1, True)
                    Move(0, <StorageType.DISK: 1>, <StorageType.FWD_RESTART: 2>)
−−−▶          

### References

[1] Andreas Griewank and Andrea Walther, 'Algorithm 799: revolve: an implementation of checkpointing for the reverse or adjoint mode of computational differentiation', ACM Transactions on Mathematical Software, 26(1), pp. 19--45, 2000, doi: 10.1145/347837.347846

[2] Herrmann, J. and Pallez (Aupy), G.. "H-Revolve: a framework for adjoint computation on synchronous hierarchical platforms." ACM Transactions on Mathematical Software (TOMS) 46.2 (2020): 1-25. doi: 10.1145/3378672

[3] Aupy, G.,  Herrmann, Ju. and Hovland, P. and Robert, Y. "Optimal multistage 
    algorithm for adjoint computation". SIAM Journal on Scientific Computing, 38(3),
    C232-C255, (2016). DOI: https://doi.org/10.1145/347837.347846